## Install JDK
## Install Spark
## Set Environment variables
## Create a Spark Session

In [1]:
!apt-get install openjdk-8-jdk -y

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

!pip install findspark

!pip install pyspark

import findspark
findspark.init()

from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").appName("Colab PySpark").getOrCreate()


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  fonts-dejavu-core fonts-dejavu-extra libatk-wrapper-java libatk-wrapper-java-jni libfontenc1
  libgail-common libgail18 libgtk2.0-0 libgtk2.0-bin libgtk2.0-common libice-dev librsvg2-common
  libsm-dev libxkbfile1 libxt-dev libxtst6 libxxf86dga1 openjdk-8-jdk-headless openjdk-8-jre
  openjdk-8-jre-headless x11-utils
Suggested packages:
  gvfs libice-doc libsm-doc libxt-doc openjdk-8-demo openjdk-8-source visualvm libnss-mdns
  fonts-nanum fonts-ipafont-gothic fonts-ipafont-mincho fonts-wqy-microhei fonts-wqy-zenhei
  fonts-indic mesa-utils
The following NEW packages will be installed:
  fonts-dejavu-core fonts-dejavu-extra libatk-wrapper-java libatk-wrapper-java-jni libfontenc1
  libgail-common libgail18 libgtk2.0-0 libgtk2.0-bin libgtk2.0-common libice-dev librsvg2-common
  libsm-dev libxkbfile1 libxt-dev libxtst6 libxxf86dga1 openjdk-

## Check if Dataframe getting created

## Copy a data file to your local Colab environment

In [2]:
!wget https://raw.githubusercontent.com/futurexskill/bigdata/master/bank_prospects.csv

--2025-01-14 04:35:34--  https://raw.githubusercontent.com/futurexskill/bigdata/master/bank_prospects.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 306 [text/plain]
Saving to: ‘bank_prospects.csv’

bank_prospects.csv  100%[===================>]     306  --.-KB/s    in 0s      

2025-01-14 04:35:35 (17.6 MB/s) - ‘bank_prospects.csv’ saved [306/306]



## Check if the file is copied

In [3]:
!ls


bank_prospects.csv  sample_data


# DataFrame

## Read the CSV file into a DataFrame

In [4]:
bankProspectsDF = spark.read.csv("bank_prospects.csv",header=True)

In [5]:
bankProspectsDF.show()

+----+------+------+-------+---------+
| Age|Salary|Gender|Country|Purchased|
+----+------+------+-------+---------+
|  18| 20000|  Male|Germany|        N|
|  19| 22000|Female| France|        N|
|  20| 24000|Female|England|        N|
|  21|  NULL|  Male|England|        N|
|  22| 50000|  Male| France|        Y|
|  23| 35000|Female|England|        N|
|  24|  NULL|  Male|Germany|        N|
|  25| 32000|Female| France|        Y|
|NULL| 35000|  Male|Germany|        N|
|  27| 37000|Female| France|        N|
|  27| 37000|Female|unknown|        N|
+----+------+------+-------+---------+



## Remove the record with unknow value in country column

In [6]:
bankProspectsDF1 = bankProspectsDF.filter(bankProspectsDF['country'] != "unknown")

In [7]:
bankProspectsDF1.show()

+----+------+------+-------+---------+
| Age|Salary|Gender|Country|Purchased|
+----+------+------+-------+---------+
|  18| 20000|  Male|Germany|        N|
|  19| 22000|Female| France|        N|
|  20| 24000|Female|England|        N|
|  21|  NULL|  Male|England|        N|
|  22| 50000|  Male| France|        Y|
|  23| 35000|Female|England|        N|
|  24|  NULL|  Male|Germany|        N|
|  25| 32000|Female| France|        Y|
|NULL| 35000|  Male|Germany|        N|
|  27| 37000|Female| France|        N|
+----+------+------+-------+---------+



##  Cast the String datatype to Integer/Float

In [8]:
bankProspectsDF1.printSchema()

root
 |-- Age: string (nullable = true)
 |-- Salary: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Purchased: string (nullable = true)



In [9]:
from pyspark.sql.types import IntegerType,FloatType

In [10]:
bankProspectsDF2 = bankProspectsDF1.withColumn("age", bankProspectsDF1["age"].cast(IntegerType())).withColumn("salary", bankProspectsDF1["salary"].cast(FloatType()))


In [11]:
bankProspectsDF2.printSchema()

root
 |-- age: integer (nullable = true)
 |-- salary: float (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Purchased: string (nullable = true)



## Replace Age and Salary with average values of their respective column

[link text](https://)import mean from sql.fuctions

In [12]:
from pyspark.sql.functions import mean

### Calculate "mean" value of the age

In [13]:
mean_age_val = bankProspectsDF2.select(mean(bankProspectsDF2['age'])).collect()

In [14]:
type(mean_age_val)

list

In [15]:
mean_age_val

[Row(avg(age)=22.11111111111111)]

In [16]:
mean_age = mean_age_val[0][0]

In [17]:
mean_age

22.11111111111111

### Calculate mean salary value

In [18]:
mean_salary_val = bankProspectsDF2.select(mean(bankProspectsDF2['salary'])).collect()

In [19]:
mean_salary = mean_salary_val[0][0]

In [20]:
mean_salary

31875.0

### Replace missing age with average value

In [21]:
bankProspectsDF2.show()

+----+-------+------+-------+---------+
| age| salary|Gender|Country|Purchased|
+----+-------+------+-------+---------+
|  18|20000.0|  Male|Germany|        N|
|  19|22000.0|Female| France|        N|
|  20|24000.0|Female|England|        N|
|  21|   NULL|  Male|England|        N|
|  22|50000.0|  Male| France|        Y|
|  23|35000.0|Female|England|        N|
|  24|   NULL|  Male|Germany|        N|
|  25|32000.0|Female| France|        Y|
|NULL|35000.0|  Male|Germany|        N|
|  27|37000.0|Female| France|        N|
+----+-------+------+-------+---------+



In [22]:
bankbankProspectsDF3 = bankProspectsDF2.na.fill(mean_age,["age"])

In [23]:
bankbankProspectsDF3.show()

+---+-------+------+-------+---------+
|age| salary|Gender|Country|Purchased|
+---+-------+------+-------+---------+
| 18|20000.0|  Male|Germany|        N|
| 19|22000.0|Female| France|        N|
| 20|24000.0|Female|England|        N|
| 21|   NULL|  Male|England|        N|
| 22|50000.0|  Male| France|        Y|
| 23|35000.0|Female|England|        N|
| 24|   NULL|  Male|Germany|        N|
| 25|32000.0|Female| France|        Y|
| 22|35000.0|  Male|Germany|        N|
| 27|37000.0|Female| France|        N|
+---+-------+------+-------+---------+



### Replace missing age with salary value

In [24]:
bankbankProspectsDF4 = bankbankProspectsDF3.na.fill(mean_salary,["salary"])

In [25]:
bankbankProspectsDF4.show()

+---+-------+------+-------+---------+
|age| salary|Gender|Country|Purchased|
+---+-------+------+-------+---------+
| 18|20000.0|  Male|Germany|        N|
| 19|22000.0|Female| France|        N|
| 20|24000.0|Female|England|        N|
| 21|31875.0|  Male|England|        N|
| 22|50000.0|  Male| France|        Y|
| 23|35000.0|Female|England|        N|
| 24|31875.0|  Male|Germany|        N|
| 25|32000.0|Female| France|        Y|
| 22|35000.0|  Male|Germany|        N|
| 27|37000.0|Female| France|        N|
+---+-------+------+-------+---------+



In [26]:
bankbankProspectsDF4.printSchema()

root
 |-- age: integer (nullable = true)
 |-- salary: float (nullable = false)
 |-- Gender: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Purchased: string (nullable = true)



## Write the transformed file to a new csv file

In [27]:
bankbankProspectsDF4.write.format("csv").save("bank_prospects_transformed")

In [28]:
!ls

bank_prospects.csv  bank_prospects_transformed	sample_data


In [29]:
!ls bank_prospects_transformed/

part-00000-e852cdaa-ea03-4a0b-aadb-8e257c36081b-c000.csv  _SUCCESS


In [31]:
!cat bank_prospects_transformed/part-00000-e852cdaa-ea03-4a0b-aadb-8e257c36081b-c000.csv

18,20000.0,Male,Germany,N
19,22000.0,Female,France,N
20,24000.0,Female,England,N
21,31875.0,Male,England,N
22,50000.0,Male,France,Y
23,35000.0,Female,England,N
24,31875.0,Male,Germany,N
25,32000.0,Female,France,Y
22,35000.0,Male,Germany,N
27,37000.0,Female,France,N
